In [1]:
# !pip install --force-reinstall -U -r requirements.txt --quiet

In [4]:
%%writefile agent1_docextraction_agent.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
################
import boto3, json
import time
import os
from logging import exception
from botocore.exceptions import ClientError
import trp
from trp import Document
from typing import Dict

# Import date class from datetime module
# from datetime import date
# from datetime import datetime
import datetime
import decimal

from typing import Optional
import traceback
import botocore
from botocore.config import Config
##################################

textract = boto3.client('textract', region_name='us-east-1')
print("Agent 1 - textract obj = ", textract)
# Define DynamoDB Database and the Dynamo Table
dynamodb_resource = boto3.resource("dynamodb")
app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
# def weather():
def doc_extraction(docid, indexid, s3files):
    """ Get doc_extraction """ # Dummy implementation
    print("inside doc_extraction if condition")
    print("inside doc_extraction receiving parameters = docid = ", docid, ", indexid =  ", indexid , ", s3files = ",s3files)
    # docid = event.get('docid')
    # docid = "DOC417927"
     # indexid = event.get('indexid')
    # indexid = "IN434221"
    print("inside doc_extraction docid = ", docid)
    # s3files = event.get('s3files')
    # s3files = "newmexicomutual/claimforms/IN434221/DOC417927/LegalCaseDocument-WC.pdf"
    print("inside doc_extraction s3files = ", s3files)
    s3bkt = "aimlusecases-pvt"

    # s3files= ["newmexicomutual/claimforms/IN311953/DOC637358/With_Claim_ID-NY_WCB–C-2F_Employers_Report_of_Work-Related_Injury.pdf"]  # For Worker Compensation
    print("s3files = ", s3files)
    start2 = time.time()  # record start time
    tbltxt,rawtext,keyvaluesText = get_doc_text([s3files])
    print("tbltxt = ", tbltxt)
    print("keyvaluesText = ", keyvaluesText)
    print("rawtext = ", rawtext)
    end2 = time.time() # record end time
    print("Time taken by extract tbltxt,rawtext,keyvaluesText = ", end2-start2, "sec")

    res = save_docs_extract(docid, indexid, s3files, str(rawtext), str(keyvaluesText), str(tbltxt))
    print(res)
    
    return {
        'statusCode': 200,
        'body': json.dumps({'rawtext': rawtext})
    }
    # return "sunny"

####################################
# COPIED FROM PREVIOUS POCs
# ---------------------------
def get_doc_text(s3files):
    tbltxt=[]
    rawtext=[]
    keyvaluesText=[]
    print('in get_doc_text()')
    jobids = resp_id(s3files)
    
    print('in get_doc_text()-> jobids', jobids)
    print('Please wait till we analyze the documents for the job ids...')
    for jobid in jobids:
        filname=list(jobid.keys())[0].split('.')[0]
        print('filname=',filname)
        jobidss=list(jobid.values())[0]
        print('jobidss=',jobidss)
        
        resanal="NotComplete"
        while(resanal=="NotComplete"):
            resanal=getJobResults(jobidss)
            
        if resanal=="FAILED":
            tabletxt="FAILED"
        else:
            tabletxt=parseresp(resanal)    
        
        response=json.dumps({"text":tabletxt})
        print('\nCOMPLETE jobidss=',jobidss)
        
        resp=json.loads(response)
        tbltxt.append({filname:resp['text'][0]})
        rawtext.append({filname:resp['text'][1]})
        keyvaluesText.append({filname:resp['text'][2]})
        
    return tbltxt,rawtext,keyvaluesText

# COPIED FROM PREVIOUS POCs
# ---------------------------
def resp_id(s3files):
#     client = boto3.client('textract')
    s3bkt = "aimlusecases-pvt"

    # Step1: Textract(Getting JobID) 
    jobids = []
    for s3PDF in s3files:
        print('s3PDF =',s3PDF)
#         s3filename = 'iassureclaim/policydocuments/' + s3PDF
        s3filename = s3PDF
        response = textract.start_document_analysis(
                DocumentLocation={
                    'S3Object': {
                        'Bucket': s3bkt,
                        'Name': s3filename
                    }
                },
                FeatureTypes=["TABLES","FORMS"],
                OutputConfig={
                    'S3Bucket': s3bkt,
                    'S3Prefix': s3PDF + '_extractedoutput.txt' #'iassureclaim/policydocuments/tblr-pdf/output'
                },
            )

        jobid=response['JobId']
        print('jobid=',jobid)
        resp = json.dumps({"text":jobid})
        print('resp=',resp)
        jobids.append({s3PDF:jobid})
    txtract_jobid=jobids
    return txtract_jobid

# COPIED FROM PREVIOUS POCs
# ---------------------------
def getJobResults(jobId):

    pages = []
    pgcnt=1
    
    response = textract.get_document_analysis(JobId=jobId,MaxResults=3) 
    pages.append(response) 
#     print('JobStatus=',response['JobStatus']) 
    
    if response['JobStatus']=="SUCCEEDED":
        print('JobStatus=',response['JobStatus']) 
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken'] 
        while(nextToken):
            pgcnt+=1
            print(pgcnt) 
            response = textract.get_document_analysis(JobId=jobId, NextToken=nextToken)
            pages.append(response)
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken'] 
        print("coming out from token formatting func")
        return pages
    elif response['JobStatus']=="FAILED":
        return("FAILED")
    else:
        return("NotComplete")


# COPIED FROM PREVIOUS POCs
# ---------------------------
def parseresp(resanal):
#     print('\nresanal=',resanal)
    doc = Document(resanal)
    prevpg=''
    tblcont=[]
    rawtext=""
    keyvaluesText=[]
    tblindex=0
    for resultPage in doc.pages: #pages of read PDF
        for Line in resultPage.lines:
            rawtext+=Line.text+"\n"
            
        for field in resultPage.form.fields:
    #         print("Key: {}, Value: {}".format(field.key, field.value))
            keyvaluesText.append("Key: {}, Value: {}".format(field.key, field.value))
        
        for table in resultPage.tables: #for each table 
            print("table #",tblindex)
            clnmtxt='Tabluar Data Processing '
            col=[]
            
            # try to get its shape - how many columns and rows
            for r, row in enumerate(table.rows):
                for c, cell in enumerate(row.cells):
                    if r==0:
                        if cell.text=="":
                            colval="description:-"
                        else:
                            colval=cell.text
                        col.append({str(c):colval})
#             print('columns',col)           
            # for each row define your format to get the data points
            for r, row in enumerate(table.rows):

                for c, cell in enumerate(row.cells):
                    if r!=0: # this is not the header row
                        clnmtxt+=col[c][str(c)]+" "+cell.text+","
                clnmtxt+=';'

            tblcont.append({tblindex:clnmtxt})
            tblindex+=1
            
#     print('\n--------------------------------------------------')
#     print('tblcont = ',tblcont)

#     print('\n--------------------------------------------------')
#     print('rawtext = ',rawtext)

#     print('\n--------------------------------------------------')
#     print('keyvaluesText = ',keyvaluesText)
            
    return tblcont, rawtext, keyvaluesText

def save_docs_extract(docid, indexid, s3filename, rawtext, keyvaluesText, tbltxt):
    print('inside save_claimform_attributes()')
    try:
        # Get the current datetime
        current_datetime = datetime.datetime.now(datetime.timezone.utc)
        # Convert the datetime to ISO 8601 format        
        sort_key = current_datetime.isoformat()
        print("sort_key (which is datetime) = ", sort_key)

        dynamodb_tbl_nm = "hcltech-doc-extraction"
        dbtable = dynamodb_resource.Table(dynamodb_tbl_nm)
        print('dbtable = ',dbtable)
        response = dbtable.put_item(
                                   Item={
                                       "docid": docid, 
                                       "indexid": indexid, 
                                       "document_name" : s3filename,
                                       "current_datetime" : str(sort_key), 
                                       "rawtext" : rawtext,   # This holds the rawtext which is used for chatbot
                                       "keyvaluesText" : keyvaluesText,   # This holds the keyvaluesText which is used for chatbot
                                       "tbltxt" : tbltxt,   # This holds the tbltxt which is used for chatbot
                                         }
                                    )

        print("successfully inserted the record in DynamoDB Table")
        return 'Saved successfully to DynamoDB'
    
    except Exception as error:
        print(f"Exception in save_docs_extract() and the error is - {error}")  
        print('Exception Details in save_docs_extract() are - ',traceback.format_exc())
        return 'Unable to save the json in DynamoDB'



########################################

model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    # tools=[calculator, weather],
    # system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
    tools=[calculator, doc_extraction],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and also perform document extraction."
)

# # @app.entrypoint
# # def strands_agent_bedrock(payload):
# #     """
# #     Invoke the agent with a payload
# #     """
# #     user_input = payload.get("prompt")
# #     print("User input:", user_input)
# #     response = agent(user_input)
# #     return response.message['content'][0]['text']

# if __name__ == "__main__":
#     print("\@@@@@@@ agent1 doc extract report:")
#     user_input = "Extract the document whose document id is DOC417927, index id is IN434221 and the s3 filepath is newmexicomutual/claimforms/IN434221/DOC417927/LegalCaseDocument-WC.pdf"
#     response = agent(user_input)
#     print("response = ", response)
#     # return response.message['content'][0]['text']
#     # app.run()

Overwriting agent1_docextraction_agent.py


In [3]:
!python agent1_docextraction_agent.py

Agent 1 - textract obj =  <botocore.client.Textract object at 0x7f8c2df5af80>
\@@@@@@@ agent1 doc extract report:
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/agentcore-samples/yaju-idp-agents/agent1_docextraction_agent.py", line 290, in <module>
    response = agent(user_input)
NameError: name 'user_input' is not defined
